In [1]:
import time

import pyspark
import networkx as nx

from les_mis import LES_MIS_GRAPH

sc = pyspark.SparkContext.getOrCreate()
graph = sc.parallelize(LES_MIS_GRAPH)

23/03/29 19:09:45 WARN Utils: Your hostname, qinning resolves to a loopback address: 127.0.1.1; using 10.0.0.182 instead (on interface wlp5s0)
23/03/29 19:09:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/29 19:09:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
print(f"Number of partitions: {str(graph.getNumPartitions())}")
print(f"First element in graph: {str(graph.first())}")

Number of partitions: 16
23/03/29 19:09:46 WARN SizeEstimator: Failed to check whether UseCompressedOops is set; assuming yes
First element in graph: ('Napoleon', 'Myriel', 1)


In [3]:
graph.first()

('Napoleon', 'Myriel', 1)

In [4]:
len(graph.collect())

254

In [5]:
all_nodes = graph.flatMap(lambda x: (x[0], x[1])).distinct().collect()
len(all_nodes)

77

In [6]:
# node_map = {node: i for i, node in enumerate(all_nodes)}
cluster_map = {f"cluster:{i}":[node] for i, node in enumerate(all_nodes)}
cluster_map

{'cluster:0': ['OldMan'],
 'cluster:1': ['Perpetue'],
 'cluster:2': ['Champmathieu'],
 'cluster:3': ['Eponine'],
 'cluster:4': ['Prouvaire'],
 'cluster:5': ['Bossuet'],
 'cluster:6': ['MotherPlutarch'],
 'cluster:7': ['Claquesous'],
 'cluster:8': ['Labarre'],
 'cluster:9': ['Marguerite'],
 'cluster:10': ['Gervais'],
 'cluster:11': ['MmeDeR'],
 'cluster:12': ['Fantine'],
 'cluster:13': ['Scaufflaire'],
 'cluster:14': ['Pontmercy'],
 'cluster:15': ['Montparnasse'],
 'cluster:16': ['Thenardier'],
 'cluster:17': ['MotherInnocent'],
 'cluster:18': ['Combeferre'],
 'cluster:19': ['Babet'],
 'cluster:20': ['Toussaint'],
 'cluster:21': ['MmeMagloire'],
 'cluster:22': ['Count'],
 'cluster:23': ['Blacheville'],
 'cluster:24': ['Jondrette'],
 'cluster:25': ['Magnon'],
 'cluster:26': ['MmePontmercy'],
 'cluster:27': ['BaronessT'],
 'cluster:28': ['Enjolras'],
 'cluster:29': ['Child2'],
 'cluster:30': ['Napoleon'],
 'cluster:31': ['Isabeau'],
 'cluster:32': ['MmeThenardier'],
 'cluster:33': ['Coset

In [7]:
# Input Cluster (C)
clusters = sc.parallelize([(cluster_map[cluster_id][0], cluster_id) for cluster_id in cluster_map.keys()], 32)

in_clusters = clusters.map(lambda x: (x[0], x))
# Input Graph (G)
encoded_edges = graph.map(lambda x: (x[0], (x[1], x[2]))).union(
    graph.map(lambda x: (x[1], (x[0], x[2])))
).sortByKey(lambda x: x[0])
len(encoded_edges.collect())

508

In [8]:

def to_list(a):
    return [a]

def append(a, b):
    a.append(b)
    return a

def extend(a, b):
    a.extend(b)
    return a

encoded_edges = encoded_edges.combineByKey(to_list, append, extend).sortBy(lambda x: x[0])

In [9]:
clusters.first()

('OldMan', 'cluster:0')

In [10]:
collected = encoded_edges.collect()
collected[0:4]

[('Anzelma', [('Eponine', 2), ('Thenardier', 2), ('MmeThenardier', 1)]),
 ('Babet',
  [('Thenardier', 6),
   ('Gueulemer', 6),
   ('Valjean', 1),
   ('MmeThenardier', 1),
   ('Javert', 2),
   ('Gavroche', 1),
   ('Eponine', 1),
   ('Claquesous', 4),
   ('Montparnasse', 2),
   ('Brujon', 3)]),
 ('Bahorel',
  [('Combeferre', 5),
   ('Gavroche', 5),
   ('Courfeyrac', 6),
   ('Mabeuf', 2),
   ('Enjolras', 4),
   ('Feuilly', 3),
   ('Prouvaire', 2),
   ('Marius', 1),
   ('Bossuet', 4),
   ('Joly', 5),
   ('Grantaire', 1),
   ('MmeHucheloup', 1)]),
 ('Bamatabois',
  [('Fantine', 1),
   ('Javert', 1),
   ('Valjean', 2),
   ('Judge', 2),
   ('Champmathieu', 2),
   ('Brevet', 1),
   ('Chenildieu', 1),
   ('Cochepaille', 1)])]

In [11]:
# make less slow :-(
zipped = in_clusters.join(encoded_edges)
zipped.collect()

[('Champmathieu',
  (('Champmathieu', 'cluster:2'),
   [('Valjean', 3),
    ('Judge', 3),
    ('Bamatabois', 2),
    ('Brevet', 2),
    ('Chenildieu', 2),
    ('Cochepaille', 2)])),
 ('Bossuet',
  (('Bossuet', 'cluster:5'),
   [('Marius', 5),
    ('Courfeyrac', 12),
    ('Gavroche', 5),
    ('Bahorel', 4),
    ('Enjolras', 10),
    ('Feuilly', 6),
    ('Prouvaire', 2),
    ('Combeferre', 9),
    ('Mabeuf', 1),
    ('Valjean', 1),
    ('Joly', 7),
    ('Grantaire', 3),
    ('MmeHucheloup', 1)])),
 ('Marguerite',
  (('Marguerite', 'cluster:9'), [('Valjean', 1), ('Fantine', 2)])),
 ('Pontmercy',
  (('Pontmercy', 'cluster:14'),
   [('Thenardier', 1), ('MmePontmercy', 1), ('Marius', 1)])),
 ('Combeferre',
  (('Combeferre', 'cluster:18'),
   [('Enjolras', 15),
    ('Marius', 5),
    ('Gavroche', 6),
    ('Mabeuf', 2),
    ('Prouvaire', 2),
    ('Feuilly', 5),
    ('Courfeyrac', 13),
    ('Bahorel', 5),
    ('Bossuet', 9),
    ('Joly', 5),
    ('Grantaire', 1)])),
 ('Babet',
  (('Babet', 'clu

In [50]:
mod_zipped = zipped.map(lambda x: (x[1][0][1], (x[0], x[1][1]))).collect()
mod_zipped[0:20]

[('cluster:2',
  ('Champmathieu',
   [('Valjean', 3),
    ('Judge', 3),
    ('Bamatabois', 2),
    ('Brevet', 2),
    ('Chenildieu', 2),
    ('Cochepaille', 2)])),
 ('cluster:5',
  ('Bossuet',
   [('Marius', 5),
    ('Courfeyrac', 12),
    ('Gavroche', 5),
    ('Bahorel', 4),
    ('Enjolras', 10),
    ('Feuilly', 6),
    ('Prouvaire', 2),
    ('Combeferre', 9),
    ('Mabeuf', 1),
    ('Valjean', 1),
    ('Joly', 7),
    ('Grantaire', 3),
    ('MmeHucheloup', 1)])),
 ('cluster:9', ('Marguerite', [('Valjean', 1), ('Fantine', 2)])),
 ('cluster:14',
  ('Pontmercy', [('Thenardier', 1), ('MmePontmercy', 1), ('Marius', 1)])),
 ('cluster:18',
  ('Combeferre',
   [('Enjolras', 15),
    ('Marius', 5),
    ('Gavroche', 6),
    ('Mabeuf', 2),
    ('Prouvaire', 2),
    ('Feuilly', 5),
    ('Courfeyrac', 13),
    ('Bahorel', 5),
    ('Bossuet', 9),
    ('Joly', 5),
    ('Grantaire', 1)])),
 ('cluster:19',
  ('Babet',
   [('Thenardier', 6),
    ('Gueulemer', 6),
    ('Valjean', 1),
    ('MmeThenardie

In [67]:
class Node:
    def __init__(self, cluster_id: str, id: str, neighbors: list[tuple[str, int]]):
        self.id = id
        self.cluster_id = cluster_id
        self.neighbors = [neighbor for neighbor, _ in neighbors]
        self.weights = [weight for _, weight in neighbors]
    
    def get_volume(self):
        return sum(self.weights)

    def get_degree(self):
        return len(self.neighbors)

    def get_neighbours(self):
        return self.neighbors

    def get_cut(self, other):
        return len(set(self.neighbors).intersection(set(other.neighbors)))


class Cluster:
    def __init__(self, id: str, nodes: list[Node]):
        self.cluster_id = id
        self.nodes = nodes
        self.node_ids = [node.id for node in nodes]
        # below might be useful later....
        self.volume = sum([node.get_volume() for node in nodes])

    def get_internal_weight(self):
        total_weight = 0
        for node in self.nodes:
            for index, edge in enumerate(node.neighbors):
                if edge in self.node_ids:
                    total_weight += node.weights[index]
        
        return total_weight / 2 # cause we do be double counting

    def get_cut_weight(self, remove_node: str) -> int:
        total_weight = 0
        for node in self.nodes:
            if node.id == remove_node:
                continue
            for index, edge in enumerate(node.neighbors):
                if edge == remove_node:
                    continue
                if edge in self.node_ids:
                    total_weight += node.weights[index]

        return total_weight / 2 # cause we do be double counting

    def cluster_modularity(self, total_weights: int):
        # Computed based on Q_c as detailed here: https://en.wikipedia.org/wiki/Louvain_method under the "Algorithm" section
        norm_in_weights = self.get_internal_weight() / (2 * total_weights)
        norm_all_weights = (self.volume / (2 * total_weights)) ** 2
        return norm_in_weights - norm_all_weights

    def add_node(self, node: Node):
        self.nodes.append(node)
        self.node_ids.append(node.id)
        self.volume += node.get_volume()

    def remove_node(self, node: Node):
        self.nodes.remove(node)
        self.node_ids.remove(node.id)
        self.volume -= node.get_volume()

    def get_cut(self, remove_node):
        return sum([node.get_cut(other_node) for node in self.nodes for other_node in remove_node.nodes])


In [64]:
node_1 = Node(
    cluster_id="cluster:1",
    id="Champmathieu",
    neighbors=[
        ('Valjean', 3),
        ('Judge', 3),
        ("Bossuet", 1), # double counting :-(
        ('Bamatabois', 2),
        ('Brevet', 2),
        ('Chenildieu', 2),
        ('Cochepaille', 2)
    ]
)
node_2 = Node(
    cluster_id="cluster:1",
    id="Bossuet",
    neighbors=[
        ('Marius', 5),
        ("Champmathieu", 1),
        ('Courfeyrac', 12),
        ('Gavroche', 5),
        ('Bahorel', 4),
        ('Enjolras', 10),
        ('Feuilly', 6),
        ('Prouvaire', 2),
        ('Combeferre', 9),
        ('Mabeuf', 1),
        ('Valjean', 1),
        ('Joly', 7),
        ('Grantaire', 3),
        ('MmeHucheloup', 1)
    ]
)
node_3 = Node(
    cluster_id="cluster:1",
    id="Combeferre",
    neighbors=[
        ('Enjolras', 15),
        ('Marius', 5),
        ('Gavroche', 6),
        ('Mabeuf', 2),
        ('Prouvaire', 2),
        ('Feuilly', 5),
        ('Courfeyrac', 13),
        ('Bahorel', 5),
        ('Bossuet', 9),
        ('Joly', 5),
        ('Grantaire', 1)
    ]
)

In [68]:
cluster = Cluster(id="cluster:1", nodes=[node_1, node_2, node_3])
cluster.get_internal_weight()

10.0

In [69]:
total_volume = sum([node.get_volume() for node in [node_1, node_2, node_3]])

cluster.cluster_modularity(total_volume)

-0.21666666666666667

In [ ]:
def compute_modularity(graph_volume: int, clusters: list[Cluster]):
    modularity = 0
    m = graph_volume
    norm = 1 / m 
    for cluster in clusters:
        modularity += (cluster.get_internal_weight() / graph_volume) - (cluster.get_cut_weight() / graph_volume) ** 2
    return modularity

In [58]:
cluster.get_cut_weight("Champmathieu")

9.0

In [ ]:
nodes = [
    (
    'Champmathieu',
    [
        ('Valjean', 3),
        ('Judge', 3),
        ('Bamatabois', 2),
        ('Brevet', 2),
        ('Chenildieu', 2),
        ('Cochepaille', 2)
    ]
   ),
    (
    'Champmathieu',
    [
        ('Valjean', 3),
        ('Judge', 3),
        ('Bamatabois', 2),
        ('Brevet', 2),
        ('Chenildieu', 2),
        ('Cochepaille', 2)
    ]
   )
]

In [19]:
mod_zipped.flatMapValues(
    lambda x: (x[0], x[1][0], reduce(map(lambda y: )))
).first()

('cluster:2', 'Champmathieu')

In [ ]:
first_agg = zipped.flatMapValues(lambda x: (x[0], )).collect()
first_agg[0:2]

In [ ]:
temp_element = ('cluster:2',
  (
    'Champmathieu',
   [
      ('Valjean', 3),
      ('Judge', 3),
      ('Bamatabois', 2),
      ('Brevet', 2),
      ('Chenildieu', 2),
      ('Cochepaille', 2)
  ]
   )
)

temp_element[1][1]

In [ ]:
# The below data structure is what we want to end up with after the Aggregate by C(luster) operation
# on page 693 of the paper. 
alternative = (
    "cluster:2",
    [
        (
            "Champmathieu",
            (
                'Valjean',
                'Judge',
                'Bamatabois',
                'Brevet',
                'Chenildieu',
                'Cochepaille'
            ),
            (
                3,
                3,
                2,
                2,
                2,
                2
            )
        ),
        # ... other elements in the cluster
    ]
)

In [ ]:
total_volume = graph.map(lambda x: x[2]).sum() / 2

cut_with_v = total_volume - sum(alternative[1][0][2])
cut_with_v

In [ ]:
print(f"First element of cluster: {str(clusters[0])}")
print(f"First encded graph edge: {str(encoded_edges.first())}")

In [ ]:
class Node:
    def __init__(self, id, neighbors):
        self.id = id
        self.neighbors = neighbors
    
    def get_volume(self):
        return len(self.neighbors)

    def get_cut(self, other):
        return len(set(self.neighbors).intersection(set(other.neighbors)))